chat-gpt에 아래처럼 물어본후 pytorch로 코드를 작성해달라고 부탁함.

입력 - 영어 : I like the boy , 출력 : 한국어 : 나는 그 소년을 좋아한다 로 번역하는 encoder-decoder 모델을 설명해주라.

하지만 버그가 있다.

참고문헌  
Encoder-Decoder Seq2Seq Models, Clearly Explained!!,   
https://medium.com/analytics-vidhya/encoder-decoder-seq2seq-models-clearly-explained-c34186fbf49b

- 다음은 PyTorch로 간단한 Encoder-Decoder 기반 번역 모델을 구현한 코드입니다.
- 이 코드에는 LSTM을 사용하며, 작은 데이터셋에서 학습을 테스트할 수 있는 구조로 작성되었습니다.

## 데이터 준비

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import numpy as np

In [20]:
# 단어 토큰화
def tokenize(sentence):
    return sentence.split()

# 단어 사전 생성
def build_vocab(sentences):
    tokens = [word for sentence in sentences for word in tokenize(sentence)]
    vocab = {word: idx for idx, word in enumerate(set(tokens), start=2)}
    vocab["<PAD>"] = 0
    vocab["<UNK>"] = 1
    return vocab

# 문장을 숫자로 변환
def encode(sentence, vocab):
    return [vocab.get(word, vocab["<UNK>"]) for word in tokenize(sentence)]

# 패딩
def pad_sequence(sequences, max_len):
    return [seq + [0] * (max_len - len(seq)) for seq in sequences]

In [13]:
# 간단한 샘플 데이터셋
data = [
    ("I like the boy", "나는 그 소년을 좋아한다"),
    ("I love apples", "나는 사과를 좋아한다"),
    ("He likes her", "그는 그녀를 좋아한다"),
]

# 데이터 준비
src_sentences = [src for src, _ in data]
tgt_sentences = [tgt for _, tgt in data]

src_vocab = build_vocab(src_sentences)
tgt_vocab = build_vocab(tgt_sentences)

src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)

src_encoded = [encode(sentence, src_vocab) for sentence in src_sentences]
tgt_encoded = [encode(sentence, tgt_vocab) for sentence in tgt_sentences]

max_src_len = max(len(seq) for seq in src_encoded)
max_tgt_len = max(len(seq) for seq in tgt_encoded)

src_padded = pad_sequence(src_encoded, max_src_len)
tgt_padded = pad_sequence(tgt_encoded, max_tgt_len)

In [25]:
src_padded

[[4, 2, 3, 9], [4, 5, 6, 0], [7, 10, 8, 0]]

## 데이터셋 및 DataLoader

In [2]:
class TranslationDataset(Dataset):
    def __init__(self, src, tgt):
        self.src = torch.tensor(src, dtype=torch.long)
        self.tgt = torch.tensor(tgt, dtype=torch.long)
    
    def __len__(self):
        return len(self.src)
    
    def __getitem__(self, idx):
        return self.src[idx], self.tgt[idx]

dataset = TranslationDataset(src_padded, tgt_padded)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [27]:
dataset[0]

(tensor([4, 2, 3, 9]), tensor([5, 2, 4, 7]))

## 모델 정의

In [3]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    
    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_size, embedding_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x, hidden, cell):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        predictions = self.fc(outputs)
        return predictions, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, src, tgt):
        hidden, cell = self.encoder(src)
        outputs, _, _ = self.decoder(tgt, hidden, cell)
        return outputs


## 학습 루프

In [4]:
# 모델 초기화
embedding_dim = 32
hidden_dim = 64

encoder = Encoder(src_vocab_size, embedding_dim, hidden_dim)
decoder = Decoder(tgt_vocab_size, embedding_dim, hidden_dim)
model = Seq2Seq(encoder, decoder)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss(ignore_index=0)  # <PAD>는 무시
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for src, tgt in dataloader:
        # 입력 및 출력
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]
        
        # 예측
        outputs = model(src, tgt_input)
        outputs = outputs.reshape(-1, tgt_vocab_size)
        tgt_output = tgt_output.reshape(-1)
        
        # 손실 계산
        loss = criterion(outputs, tgt_output)
        
        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(dataloader):.4f}")


Epoch [1/10], Loss: 2.1663
Epoch [2/10], Loss: 2.1114
Epoch [3/10], Loss: 2.0438
Epoch [4/10], Loss: 2.0165
Epoch [5/10], Loss: 1.9745
Epoch [6/10], Loss: 1.8946
Epoch [7/10], Loss: 1.8402
Epoch [8/10], Loss: 1.8217
Epoch [9/10], Loss: 1.7718
Epoch [10/10], Loss: 1.6588


## 예측 테스트

In [6]:
def translate(sentence, model, src_vocab, tgt_vocab, max_len=10):
    model.eval()
    src_encoded = encode(sentence, src_vocab)
    src_tensor = torch.tensor([src_encoded], dtype=torch.long)
    
    hidden, cell = model.encoder(src_tensor)
    tgt_input = torch.tensor([[tgt_vocab["<START>"]]], dtype=torch.long)
    
    translation = []
    for _ in range(max_len):
        output, hidden, cell = model.decoder(tgt_input, hidden, cell)
        pred = output.argmax(2)[:, -1].item()
        if pred == tgt_vocab["<END>"]:
            break
        translation.append(pred)
        tgt_input = torch.tensor([[pred]], dtype=torch.long)
    
    rev_tgt_vocab = {idx: word for word, idx in tgt_vocab.items()}
    return " ".join(rev_tgt_vocab[idx] for idx in translation)

# 테스트
test_sentence = "I like the boy"
result = translate(test_sentence, model, src_vocab, tgt_vocab)
print(f"Input: {test_sentence}")
print(f"Translation: {result}")


KeyError: '<START>'

## 요약 
위 코드는 간단한 Encoder-Decoder 구조로 번역을 구현합니다. 더 나은 성능을 위해서는 어텐션 메커니즘, 더 큰 데이터셋, 또는 Transformer 구조를 사용할 수 있습니다.